In [1]:
import sys
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose')
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src')
#sys.path.append('/mnt/c/Users/Jesse/Desktop/glucose/src/policies')
sys.path.append('/users/jclih/glucose')
sys.path.append('/users/jclih/glucose/src')
sys.path.append('/users/jclih/glucose/policies')

In [2]:
from policies.policies import model_smoothed_fitted_q
from Glucose import Glucose
from transition_estimation import MultivariateLinear
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [ ]:
env = Glucose(horizon=50)
transition_model_fitter = MultivariateLinear
gamma = 0.9
number_of_value_iterations = 1

In [ ]:
done = False
env.reset()
while not done:
    action, pairwise_kernel_, kernel_sums = \
        model_smoothed_fitted_q(env, gamma, RandomForestRegressor, number_of_value_iterations, transition_model_fitter,
                                pairwise_kernels_=None, kernel_sums=None)
    _, r, done = env.step(action)

/users/jclih/glucose/src/transition_estimation.py:212: RuntimeWarning: covariance is not positive-semidefinite.
  y_simulated = np.random.multivariate_normal(mean, cov=self.sigma_hat)
/users/jclih/glucose/src/transition_estimation.py:172: RuntimeWarning: divide by zero encountered in divide
  self.sigma_hat = np.cov(errors_array.T) / (n - p)


> /users/jclih/glucose/src/transition_estimation.py(223)simulate_from_fit_model_at_block()
-> Sp1.append(sp1), Xp1.append(xp1)
(Pdb) sp1
*** NameError: name 'sp1' is not defined
(Pdb) print(x)
[  1.         231.61125305  -0.           4.20477625 231.61125305
  -0.           4.20477625   0.           0.        ]
(Pdb) self.sigma_hat
array([[ inf, -inf,  inf],
       [-inf,  inf, -inf],
       [ inf, -inf,  inf]])
